In [5]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup

API key, Petr: 24a6e38c235025362537c2bcd85e66d9

In [12]:
class LastFMDownloader: 
    '''
    Downloader class for collection of data and storage of results.
    '''
    def __init__(self, allowLog = True):
        '''
        Initilization of Downloader object. Storing objects from the webpage with self.qwe, self.rty and self.uio
        '''
        self.allowLog = allowLog
        if self.allowLog:
            print('Downloader initialized.')
            
    def GetSoup(self, link):
        '''
        Returns soup in xml of from the given link in utf-8 encoding.
        '''
        self.link = link
        r = requests.get(link)
        r.encoding='UTF-8'
        self.soup = BeautifulSoup(r.text,'lxml')
    
    def GetJson(self, link):
        '''
        Returns JSON file for a given request
        '''
        self.linkToJson = link
        self.json = requests.get(link).json()
        
    
    def GetEmArtists(self, jsn):
        self.jsnPd = pd.read_json(json.dumps(jsn))
        self.topArtists = pd.DataFrame(columns=['Country', 'Artist', 'Scrobbles'])
        for i in range(1000):
            try:
                hlp = pd.DataFrame([[self.jsnPd.iloc[0,:][0]['country'],self.jsnPd.iloc[1,:][0][i]['name'],self.jsnPd.iloc[1,:][0][i]['listeners']]], columns=['Country', 'Artist', 'Scrobbles'])
                self.topArtists = self.topArtists.append(hlp)                
            except:
                pass
        return self.topArtists       
    
    def ProcessTheArtists(self, jsonFile):
        '''
        Process json files for artist info
        '''
        dfc = pd.read_json(json.dumps)
        
    
    def ParseSoupTag(self, tag):
        '''
        Returns a list of all lines with a certain tag.
        '''
        return self.soup.findAll(tag)
    
    def ParseSoupTagAttribute(self, tag, attribute, attribute_value):
        '''
        Returns a list of all lines with a certain tag given it's attribute.
        '''
        return self.soup.findAll(tag, {attribute, attribute_value})
    
    def SaveDfs(self): 
        '''
        Storing objects from the webpage as pandas DataFrame dictionary.
        '''
        dfs = {}
        dfs['Artists'] = self.artists
        dfs['Albums'] = self.albums
        dfs['Tracks'] = self.tracks
        self.resultsdf = dfs
        return self.resultsdf
    
    def getlinks(self):
        '''
        Returns all links.
        '''
        all_links = self.soup.find_all('a')
        for link in all_links:
             self.links = print(link.get('href'))
        return self.links
        


In [13]:
states = pd.read_csv("countries_cut_one.csv", sep =";", encoding='latin-1')
rqst_artist = 'https://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&limit=1000&country=' + states + '&api_key=a96a3d9d79f3af63343555c8ba1451f6&format=json'

rqst_artist['Name'] = states['NAME']
rqst_artist.head()


,NAME,COUNTRY,Name
0,https://ws.audioscrobbler.com/2.0/?method=geo....,https://ws.audioscrobbler.com/2.0/?method=geo....,Taiwan
1,https://ws.audioscrobbler.com/2.0/?method=geo....,https://ws.audioscrobbler.com/2.0/?method=geo....,Afghanistan
2,https://ws.audioscrobbler.com/2.0/?method=geo....,https://ws.audioscrobbler.com/2.0/?method=geo....,Albania
3,https://ws.audioscrobbler.com/2.0/?method=geo....,https://ws.audioscrobbler.com/2.0/?method=geo....,Algeria
4,https://ws.audioscrobbler.com/2.0/?method=geo....,https://ws.audioscrobbler.com/2.0/?method=geo....,American Samoa


In [14]:
Countries = []

for k in range(len(rqst_artist)):
    new_state = LastFMDownloader()    
    print(k) # knowing the stage
    new_state.GetJson(rqst_artist.iloc[k]['COUNTRY'])    
    Countries.append(new_state)
    

Downloader initialized.
0
Downloader initialized.
1
Downloader initialized.
2
Downloader initialized.
3
Downloader initialized.
4
Downloader initialized.
5
Downloader initialized.
6
Downloader initialized.
7
Downloader initialized.
8
Downloader initialized.
9
Downloader initialized.
10
Downloader initialized.
11
Downloader initialized.
12
Downloader initialized.
13
Downloader initialized.
14
Downloader initialized.
15
Downloader initialized.
16
Downloader initialized.
17
Downloader initialized.
18


KeyboardInterrupt: 

[<__main__.LastFMDownloader object at 0x000001B0EA2820F0>, <__main__.LastFMDownloader object at 0x000001B09AA2B128>, <__main__.LastFMDownloader object at 0x000001B09AA2B5C0>, <__main__.LastFMDownloader object at 0x000001B0F9014048>, <__main__.LastFMDownloader object at 0x000001B0F95560F0>, <__main__.LastFMDownloader object at 0x000001B085A96208>, <__main__.LastFMDownloader object at 0x000001B091433240>, <__main__.LastFMDownloader object at 0x000001B09AA06898>, <__main__.LastFMDownloader object at 0x000001B0929CFC50>, <__main__.LastFMDownloader object at 0x000001B09AA06208>, <__main__.LastFMDownloader object at 0x000001B0F9014978>, <__main__.LastFMDownloader object at 0x000001B0929CFA90>, <__main__.LastFMDownloader object at 0x000001B0FA9EABE0>, <__main__.LastFMDownloader object at 0x000001B081B6CFD0>, <__main__.LastFMDownloader object at 0x000001B081B86278>, <__main__.LastFMDownloader object at 0x000001B0924BABE0>, <__main__.LastFMDownloader object at 0x000001B0F7B61BA8>, <__main__.Las

In [16]:
for k in range(len(Countries)):
    #Countries[k].ParseSoupTag('name')    
    Countries[k].GetEmArtists(Countries[k].json)
          


In [ ]:
Countries[1].topArtists


In [25]:
portugal = LastFMDownloader()
portugal.GetJson('http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&limit=1000&country=portugal&api_key=24a6e38c235025362537c2bcd85e66d9&format=json')

Downloader initialized.


In [26]:
portugal.GetEmArtists(portugal.json)

,Country,Artist,Scrobbles
0,Portugal,Radiohead,4820265
0,Portugal,David Bowie,3442359
0,Portugal,Arctic Monkeys,3593534
0,Portugal,Queen,4138277
0,Portugal,The Beatles,3782432
0,Portugal,Arcade Fire,2618374
0,Portugal,Tame Impala,1244852
0,Portugal,Pink Floyd,3166648
0,Portugal,Nirvana,4361153
0,Portugal,The xx,1973563


class - artist